<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex15notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex15notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 多変量解析から機械学習へ − 識別問題の例 −
---

「多変量解析及び演習」に続く科目はいくつかありますが，ここでは「機械学習I/II」（3年1Q2Q）や「機械学習特論I/II」（大学院1Q2Q）等で学ぶ内容を少しだけ紹介します．
これらの授業の一部の回では，この授業の「判別分析」の回に学んだ識別問題，すなわち，与えられたデータを予め定めたいくつかのクラスに分類する問題を扱います．

<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scipy の多変量正規分布
from scipy.stats import multivariate_normal

### 機械学習ライブラリ scikit-learn のいろいろ
#
# データの準備他いろいろ
from sklearn.datasets import make_moons, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# 二次判別分析
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# 混合正規分布モデル(Gaussian Mixture Model)
from sklearn.mixture import GaussianMixture
# 階層型ニューラルネットワーク
from sklearn.neural_network import MLPClassifier

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 判別分析ではうまくいかない例




判別分析は，「クラスごとのデータが正規分布に従っている」という仮定をおく手法ですので，データがこの条件を満たさない場合は良い結果を得られないことがあります．
「判別分析 (2)」の回の notebookB「判別分析に関する補足」で使ったのと同じようなデータでもう一度実験してみましょう．

#### データの準備

In [ ]:
# 例題のデータを生成
K = 2
NL, NT = 500, 500
X_moon, Y_moon = make_moons(n_samples=NL+NT, noise=0.2)
XL, yL = X_moon[:NL, :], Y_moon[:NL]
XT, yT = X_moon[NL:, :], Y_moon[NL:]
print(XL.shape, yL.shape)
print(XT.shape, yT.shape)

In [ ]:
# 散布図を表示
xmin, xmax = -2, 3
ymin, ymax = -2.5, 2.5
fig = plt.figure(figsize=(9, 6))
ax0 = fig.add_subplot(121)
ax0.scatter(XL[yL==0, 0], XL[yL==0, 1], marker='.', label='Class0')
ax0.scatter(XL[yL==1, 0], XL[yL==1, 1], marker='.', label='Class1')
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')
ax0.legend()
ax0.set_title('training data')
ax1 = fig.add_subplot(122)
ax1.scatter(XT[yT==0, 0], XT[yT==0, 1], marker='.', label='Class0')
ax1.scatter(XT[yT==1, 0], XT[yT==1, 1], marker='.', label='Class1')
ax1.set_xlim(-2, 3)
ax1.set_ylim(-2.5, 2.5)
ax1.set_aspect('equal')
ax1.legend()
ax1.set_title('test data')
plt.show()

左図が学習データ（パラメータの推定に用いるデータ），右図がテストデータ（得られた識別モデルの性能評価に用いるデータ）を示します．クラスごとのデータが明らかに正規分布に従っていません．

#### 二次判別分析

上記のデータに二次判別分析を適用してみると...

In [ ]:
# 学習データを用いて二次判別分析モデルのパラメータを推定する
#   ＝ クラスごとのデータに正規分布を当てはめる
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(XL, yL)
mu = qda.means_
cov = np.array(qda.covariance_)
print(mu.shape, cov.shape)

In [ ]:
# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# 尤度の計算
p = qda.predict_proba(X_mesh.reshape((-1, 2))).T
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(10, 6))

# 左図
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(XL[yL==k, 0], XL[yL==k, 1], marker='.')
    z = multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k])
    ax0.contour(x_mesh, y_mesh, z, levels=4)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 右図
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(XL[yL==k, 0], XL[yL==k, 1], marker='.')
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

# 学習データ/テストデータの正答率
print(f'学習データの正答率: {qda.score(XL, yL)}')
print(f'テストデータの正答率: {qda.score(XT, yT)}')

左図は，推定された正規分布を学習データに重ねて表示しています．右図は，推定された正規分布を用いて計算した尤度の値を可視化しています．
図の下の数値は，識別の正答率（accuracy）を表しています．

予想通り，クラス毎のデータに正規分布がうまく当てはまらないため，2クラスをうまく分けることができていません．

##### 問題1

次の箱の中に入る数を答えなさい．

この実験の場合，データの分布を表すための正規分布の平均は $\fbox{?}$ 次元ベクトルで，分散共分散行列は $\fbox{?} \times \fbox{?}$ 行列である．推定される平均は全部で $\fbox{?}$個，推定される分散共分散行列は全部で $\fbox{?}$ 個である．

In [ ]:
# このセルを実行すると，上記の問に対する解答例が表示されます
Q = b'CuWFqOOBpiAyCg=='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

#### 混合正規分布モデル



上で見たように，正規分布は単純すぎてうまくデータの分布を表現できないことがあります．
そういう場合への対応の仕方はいろいろありますが，ここでは，「複数の正規分布の重み付き和」で表される「混合正規分布モデル」(Gaussian mixture model)を紹介します（注）．

これは，$M$個の正規分布の重み付き和で，次のように表されます．

$$
p(\pmb{x}) = \sum_{m=1}^{M} w_m N(\pmb{x}|\pmb{\mu}_m, \Sigma_m) \qquad (1)
$$

$N(\pmb{x}|\pmb{\mu}_m, \Sigma_m)$ は平均 $\pmb{\mu}_m$ 分散共分散行列 $\Sigma_m$ の多変量正規分布です．$w_m$ は $m$ 番目の正規分布の重みを表し，$0 \leq w_m \leq 1, \sum_{m=1}^M w_m = 1$ を満たします．
混合正規分布モデルでは，$w_m, \pmb{\mu}_m, \Sigma_m$ がモデルパラメータとなります．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「混合正規分布モデル」については，大学院科目「機械学習I/II」で学べるかも．</span>

試しに，上記のデータに $M=3$ の混合正規分布モデルを当てはめて識別する実験をやってみましょう．クラスごとのデータに 式$(1)$ 混合正規分布を当てはめます．
詳しくは説明しませんが，判別分析のときと同様に，尤度の大小によってクラスを識別することができます．

次のコードセルを実行すると，データに混合正規分布モデルを当てはめる計算を行います．

In [ ]:
M = 3 # クラスごとの GMM の混合数（正規分布の数）

N, D = XL.shape

# 事前確率 = クラスごとのデータの出現確率を推定
py = np.empty(K)
for k in range(K):
    py[k] = np.sum(yL == k) / NL

# クラスごとのデータの分布をGMMでモデル化
mu = np.empty((K, M, D))
cov = np.empty((K, M, D, D))
gmm = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[yL==k, :]
    gmm[k] = GaussianMixture(n_components=M, covariance_type='full', verbose=2, verbose_interval=1)
    gmm[k].fit(XX)
    mu[k, ::] = gmm[k].means_
    cov[k, ::] = gmm[k].covariances_

次のコードセルを実行すると，当てはめの結果と，それを用いて2次元平面上の各点を2クラスに識別した結果を可視化します．

In [ ]:
# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 尤度の計算
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    proba = np.exp(gmm[k].score_samples(X_mesh.reshape((-1, 2))))
    p[k, :] = py[k] * proba
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(10, 6))

# Gaussian を当てはめた結果
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(XL[yL==k, 0], XL[yL==k, 1], marker='.')
    for m in range(M):
        z = multivariate_normal.pdf(X_mesh, mean=mu[k, m], cov=cov[k, m])
        ax0.contour(x_mesh, y_mesh, z, levels=4)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 尤度の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(XL[yL==k, 0], XL[yL==k, 1], marker='.')
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

# 学習データの識別
N = XL.shape[0]
LL = np.empty((N, K))
for k in range(K):
    LL[:, k] = gmm[k].score_samples(XL) # 対数尤度
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(yL == Yp)
print(f'学習データの正答率: {ncorrect/N}')

# テストデータの識別
N = XT.shape[0]
LL = np.empty((N, K))
for k in range(K):
    LL[:, k] = gmm[k].score_samples(XT)
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(yT == Yp)
print(f'テストデータの正答率: {ncorrect/N}')

学習データテストデータともに，単一の正規分布を当てはめたとき（二次判別分析を適用したとき）よりも高い正答率が得られています（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: データに単一の正規分布を当てはめる最尤推定では，一撃の計算で最適なパラメータが求まります．しかし，混合正規分布モデルの場合，パラメータの推定は，適当な初期値からより尤度の大きいパラメータを求める計算を繰り返して徐々に最適化していく手続きとなります．そのため，上記のセルの実行結果は，セルを実行し直すたびに少し変わり得ます．

`M = 3` のところの数を変えると，正規分布の数を変えられます．いろいろ変えてみるとよいでしょう（あまり多くするとエラーになるかもしれません）．

##### 問題2

次の箱の中に入る数を答えなさい．

この実験で $M = 3$ とした場合，推定すべき平均は全部で $\fbox{?}$個，推定すべき分散共分散行列は全部で $\fbox{?}$ 個である．

In [ ]:
# このセルを実行すると，上記の問に対する解答例が表示されます
Q = b'Ck0gPSAzIOOBrua3t+WQiOato+imj+WIhuW4g+ODouODh+ODq+OBq+OBr++8jDPjgaTjga7lubPlnYfjgajliIbmlaPlhbHliIbmlaPooYzliJfjgYzjgYLjgorjgb7jgZnvvIjjgZ3jga7ku5bvvIzph43jgb8gJHdfMSwgd18yLCB3XzMkIOOCguaOqOWumuOBmeOBueOBjeODkeODqeODoeODvOOCv+OBqOOBquOCiuOBvuOBme+8ie+8jgrjgZPjga7lrp/pqJPjga8y44Kv44Op44K56K2Y5Yil5ZWP6aGM44Gq44Gu44Gn77yMMuOBpOOBruOCr+ODqeOCueOBruODh+ODvOOCv+OBq+OBneOCjOOBnuOCjOa3t+WQiOato+imj+WIhuW4g+ODouODh+ODq+OCkuW9k+OBpuOBr+OCgeOBvuOBme+8jgrjgZfjgZ/jgYzjgaPjgabvvIzmjqjlrprjgZnjgbnjgY3lubPlnYfjga/lhajpg6jjgacgJDIgXHRpbWVzIDMgPSA2JCDlgIvvvIzliIbmlaPlhbHliIbmlaPooYzliJfjgoLlhajpg6jjgacgJDYkIOWAi+OBguOCiuOBvuOBme+8jgo='
display(Markdown(base64.b64decode(Q).decode('utf-8')))

#### ニューラルネットワーク


次は，ニューラルネットワークの例です．ニューラルネットワークは，こんにちのAIの中核となっている機械学習モデルです．
ここではその詳細については省略します（注）．

次のコードセルを実行すると，データを2クラスに分けるニューラルネットワークモデルのパラメータを推定します．その計算は，与えられたデータをモデルに入力して予測値を求め，予測値と正解との間のずれに応じてパラメータを修正する作業の繰り返しです．次のセルを実行して表示される `loss` の値は，学習データに対するそのずれの大きさを表します．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 「ニューラルネットワーク」については，3年次科目「機械学習I/II」で学べるかも．</span>

In [ ]:
# ニューラルネットワークモデルの作成
neunet = MLPClassifier(hidden_layer_sizes=[500, 500], activation='relu', verbose=True)
# 学習（パラメータの推定）
neunet.fit(XL, yL)

In [ ]:
# グラフ描画用のグリッドデータの作成
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# 事後確率の推定
p = neunet.predict_proba(X_mesh.reshape((-1, 2))).T
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(4, 4))

# 事後確率の可視化
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(111)
for k in range(K):
    ax1.scatter(XL[yL==k, 0], XL[yL==k, 1], marker='.')
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

# 学習データ/テストデータの正答率
print(f'学習データの正答率: {neunet.score(XL, yL)}')
print(f'テストデータの正答率: {neunet.score(XT, yT)}')

混合正規分布モデルの場合と同様に，2つのクラスをそれなりにうまく分けることができているようです．

---
### 手書き数字識別




以前扱った手書き数字のデータと同様のものを用いて，10クラスの画像識別の実験をやってみましょう．

#### データの準備

「判別分析 (2)」の回の notebookC「演習課題: 手書き数字識別」で扱ったデータは，MNIST と呼ばれる有名な手書き数字データセット（全部で6万枚の数字画像から成る）から 6000 枚を取り出したものでしたが，ここでは2万枚を取り出して使います．

In [ ]:
# MNIST データセットの入手
Xraw, yraw = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True, as_frame=False)
Xall = Xraw[:20000] / 255.0     # 画素値が [0, 255] の整数値なので [0, 1] の浮動小数点数値に変換
yall = yraw[:20000].astype(int) # クラスラベル．0 から 9 の整数値

# 学習データとテストデータの分割
X_train, X_test, y_train, y_test = train_test_split(Xall, yall, test_size=4000, random_state=4649, stratify=yall)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
N_train, N_test = len(X_train), len(X_test)

K = 10

2万枚のうち16000枚を学習（パラメータの推定）用にし，残り4000枚をテスト（性能の評価）用にします．

In [ ]:
# 学習データの一部を可視化
nrow, ncol = 4, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(8, 4))
for i in range(nrow):
    for j in range(ncol):
        n = i * ncol + j
        ax[i, j].imshow(X_train[n].reshape(28, 28), vmin=0, vmax=1, cmap='gray')
        ax[i, j].axis('off')
        ax[i, j].set_title(f'{y_train[n]}')
fig.tight_layout()
plt.show()

#### 二次判別分析

まずは二次判別分析．

In [ ]:
# 学習データを用いて二次判別分析のパラメータを推定
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_train, y_train)

# 推定されたパラメータの shape を表示
mu = qda.means_
cov = np.array(qda.covariance_)
print(f'mu.shape: {mu.shape}')
print(f'cov.shape: {cov.shape}')

# 平均を可視化
fig, ax = plt.subplots(1, 10, figsize=(8, 1))
for k in range(10):
    ax[k].imshow(mu[k].reshape(28, 28), vmin=0, vmax=1, cmap='gray')
    ax[k].axis('off')
    ax[k].set_title(f'{k}')
fig.tight_layout()
plt.show()

上の画像は，クラスごとのデータの平均を可視化したものです．
推定されたパラメータを使って識別させてみて，その正答率を求めると，次のようになります．

In [ ]:
# 正答率
print(f'学習データの正答率   = {qda.score(X_train, y_train)}')
print(f'テストデータの正答率 = {qda.score(X_test, y_test)}')
print()
# 混同行列  confusion[i, j] は，正解がクラス i で予測がクラス j だったものの数
y_pred = qda.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

以前の実験よりもデータ数を増やしたので，次元削減しなくても一応それなりの結果が出ました．

#### 混合正規分布モデル

次は混合正規分布モデルを用いた実験です．2次元データの例では，混合正規分布モデルに含まれる正規分布の分散共分散行列の形には何ら制約はありませんでした．しかし，こちらのデータは次元数が大きい（784次元）ので，同じようにすると，推定しなければならないパラメータ数が膨大になり，パラメータをうまく推定できなくなる心配があります．そのため，ここでは，全ての分散共分散行列が対角行列であるとする制約を課すことにします．これによって，制約すべきパラメータ数を大きく減らすことができます（注）．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 次元数 $D$ のとき，制約のない分散共分散行列のひとつのパラメータ数は $\frac{1}{2}D(D+1)$ 個，対角行列に限定される場合は $D$ 個．
</span>

In [ ]:
# 学習
M = 3

py = np.empty(K)
gmm = np.empty(K, dtype=object)
for k in range(K):
    print(f'##### class{k} #####')
    py[k] = np.sum(y_train == k) / NL
    # covariance_type = 'diag' は，分散共分散行列を対角行列に限定．`full` だと任意
    gmm[k] = GaussianMixture(n_components=M, covariance_type='diag', verbose=2, verbose_interval=1)
    gmm[k].fit(X_train[y_train == k, :])

# 平均の可視化
fig, ax = plt.subplots(M, K, figsize=(8, 8/K*M))
for k in range(K):
    for m in range(M):
        img = gmm[k].means_[m, :].reshape((28, 28))
        ax[m, k].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[m, k].axis('off')
plt.show()

##### 問題3

上の画像は，クラスごとの学習データに混合正規分布モデルを当てはめて得られる平均を可視化したものです．この画像の枚数が表示されたようになる理由を考えなさい．


In [ ]:
# 学習データの識別
LL = np.empty((N_train, K))
for k in range(K):
    LL[:, k] = gmm[k].score_samples(X_train) + np.log(py[k])
y_pred = np.argmax(LL, axis=1)
countL = np.sum(y_pred == y_train)

# テストデータの識別
LT = np.empty((N_test, K))
for k in range(K):
    LT[:, k] = gmm[k].score_samples(X_test) + np.log(py[k])
y_pred = np.argmax(LT, axis=1)
countT = np.sum(y_pred == y_test)

# 結果の表示
print(f'M = {M}   train: {countL}/{N_train} = {countL/N_train:.3f}   test: {countT}/{N_test} = {countT/N_test:.3f}')
# 混同行列  confusion[i, j] は，正解がクラス i で予測がクラス j だったものの数
confusion = confusion_matrix(y_test, y_pred)
print(confusion)

混合正規分布モデルを用いると，二次判別分析よりよい結果が得られるようです．`M` の値をいろいろ変えて実験してみましょう．

#### ニューラルネットワーク

次はニューラルネットワークの例です．次のコードセルを実行すると，ニューラルネットワークを学習（モデルパラメータを修正する作業を何度も繰り返す）させます．データの数も次元数も大きいので，数分かかります．

In [ ]:
# ニューラルネットワークモデルの作成
neunet = MLPClassifier(hidden_layer_sizes=[1000, 1000], activation='relu', verbose=True)
# 学習（パラメータの推定）
neunet.fit(X_train, y_train)

二次判別分析や混合正規分布モデルを用いた手法と比べて，正答率はどう違うでしょうか．

In [ ]:
# 正答率
print(f'学習データの正答率   = {neunet.score(X_train, y_train)}')
print(f'テストデータの正答率 = {neunet.score(X_test, y_test)}')
print()
# 混同行列  confusion[i, j] は，正解がクラス i で予測がクラス j だったものの数
y_pred = neunet.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
print(confusion)